The objective of this notebook is to cross-validate the 2 different NCBI blast mappings meant to identify viral contigs with read-level classifications produced by Kraken

Contigs will be filtered to only those that are classified as viral by NCBI blast against the "ref_viruses_rep_genomes" blast database using 2 different algorithms
- blastn
    - this is expected to be overly generous
    - any contig NOT identified by this algorithm as being potentially viral is dropped automatically, but we expect more false positives than false negatives
- dc-megablast
    - this is expected to be close to correct
    - there may still be some false positives in this

In [ ]:
# if super busted, shut down kernel, remove ~/.julia, start a REPL and reinstall IJulia, then restart here

In [1]:
if isfile("Project.toml")
    println("removing Project.toml...")
    rm("Project.toml")
end
if isfile("Manifest.toml")
    println("removing Manifest.toml...")
    rm("Manifest.toml")
end
ENV["LD_LIBRARY_PATH"] = ""

# delete baseline environment???

import Pkg

pkgs = [
"Dates",
"BioSequences",
"DataFrames",
"FASTX",
"Statistics",
"StatsPlots",
"uCSV",
"Revise",
"Kmers",
"StatsBase",
"ProgressMeter",
"XAM"
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

import Mycelia

    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
  No Changes to `/global/cfs/cdirs/m4269/cjprybol/Mycelia/Project.toml`
  No Changes to `/global/cfs/cdirs/m4269/cjprybol/Mycelia/Manifest.toml`
[ Info: Precompiling Mycelia [453d265d-8292-4a7b-a57c-dce3f9ae6acd]
[ Info: Skipping precompilation since __precompile__(false). Importing Mycelia [453d265d-8292-4a7b-a57c-dce3f9ae6acd].


KeyError: KeyError: key "usage_request" not found

In [2]:
data_dir = joinpath(dirname(pwd()), "data")

"/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/viral-pangenome-discovery/data"

In [3]:
blastdbs_dir = "$(homedir())/workspace/blastdbs"
taxdump_dir = mkpath(joinpath(blastdbs_dir, "taxdump"))
taxdump_tar = joinpath(taxdump_dir, "taxdump.tar.gz")
if !isfile(taxdump_tar)
    run(`wget --quiet https://ftp.ncbi.nih.gov/pub/taxonomy/taxdump.tar.gz --directory-prefix=$(taxdump_dir)`)
end
if isempty(filter(x -> occursin(r"\.dmp$", x), readdir(taxdump_dir)))
    run(`tar -xvzf $(taxdump_tar) --directory $(taxdump_dir)`)
end

In [4]:
SRR_paths = filter(x -> !occursin(".ipynb_checkpoints", x), readdir(joinpath(data_dir, "SRA"), join=true))
SRR_paths = filter(x -> isfile(joinpath(x, "megahit", "final.contigs.fastg.gfa.fna")), SRR_paths)

# get all taxonids at or below virus
# mamba create -n taxonkit -c bioconda taxonkit
# wget -c ftp://ftp.ncbi.nih.gov/pub/taxonomy/taxdump.tar.gz 
# tar -zxvf taxdump.tar.gz
# mkdir -p $HOME/.taxonkit
# cp names.dmp nodes.dmp delnodes.dmp merged.dmp $HOME/.taxonkit
# --data-dir
viral_tax_ids = Set(parse.(Int, filter(!isempty, readlines(`mamba run -n taxonkit taxonkit list --ids 10239 --indent ""`))))
n_methods = 8
data_dir = joinpath(dirname(pwd()), "data")
RESULTS_DIR = mkpath(joinpath(data_dir, "results"))

# SRR_paths = filter(x -> !occursin(".ipynb_checkpoints", x), readdir(joinpath(data_dir, "SRA"), join=true))

"/global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/viral-pangenome-discovery/data/results"

KeyError: KeyError: key "usage_request" not found

In [ ]:
viral_blast_hits_joint_table = DataFrames.DataFrame()
ProgressMeter.@showprogress for SRR_path in SRR_paths
    # CONTIG INFO TABLE HAS JUST METADATA ABOUT THE CONTIG AND THE READS MAPPING BACK TO IT
    contig_info_pattern = basename(SRR_path) * ".final.contigs.fastg.gfa.fna"
    contig_info_files = filter(x -> occursin(contig_info_pattern, x), readdir(SRR_path, join=true))
    joint_contig_info_table = DataFrames.DataFrame()
    for f in contig_info_files
        table_col_types = [
            Int64,
            Int64,
            Int64,
            Float64,
            Float64,
            Float64,
            String,
            String,
            String,
            Int64,
            String,
            Float64,
            Float64,
            Float64,
            Float64,
            Int64,
            Int64,
            Int64,
            Int64,
            Int64,
            Int64,
            Int64,
            Int64,
            Int64,
        ]
        method = replace(replace(basename(f), basename(SRR_path) * ".final.contigs.fastg.gfa.fna." => ""), ".contig_info.tsv" => "")
        this_contig_info_table = DataFrames.DataFrame(uCSV.read(f, delim='\t', header=1, types=table_col_types, encodings=Dict("" => missing), allowmissing=true)...)
        this_contig_info_table[!, "Method"] .= method
        this_contig_info_table[!, "SRR"] .= basename(SRR_path)
        append!(joint_contig_info_table, this_contig_info_table)
    end
    sort!(joint_contig_info_table, "Contig")
    contig_info_table = unique(joint_contig_info_table[!, ["SRR", "Contig", "Length", "Mapped bases", "Mean coverage", "Standard Deviation", "% Mapped bases"]])

    # BLAST TOP HITS TABLE HAS TOP HIT FOR EACH CONTIG FOR EACH METHOD
    blast_classifications_table = joint_contig_info_table[.!ismissing.(joint_contig_info_table[!, "subject id"]), DataFrames.Not(names(contig_info_table[!, DataFrames.Not("Contig")]))]
    sort!(blast_classifications_table, ["Contig", "evalue"])
    blast_hits_top_hits_table = DataFrames.combine(DataFrames.groupby(blast_classifications_table, ["Contig", "Method"]), first)
    blast_hits_top_hits_table = blast_hits_top_hits_table[map(x -> x in viral_tax_ids, blast_hits_top_hits_table[!, "subject tax id"]), :]

    # JOINT LCA TABLE HAS CLASSIFICATION FOR EACH MMSEQS RUN
    joint_lca_table = DataFrames.DataFrame()
    easy_taxonomy_lca_reports = filter(x -> occursin("final.contigs.fastg.gfa.fna.mmseqs_easy_taxonomy.", x) && occursin("_lca.tsv", x), readdir(joinpath(SRR_path, "mmseqs_easy_taxonomy"), join=true))
    for lca_tsv in easy_taxonomy_lca_reports
        method = replace(replace(basename(lca_tsv), "final.contigs.fastg.gfa.fna." => ""), "_lca.tsv" => "")
        this_lca_table = Mycelia.parse_mmseqs_easy_taxonomy_lca_tsv(lca_tsv)
        this_lca_table[!, "SRR"] .= basename(SRR_path)
        this_lca_table[!, "SRR"] .= method
        append!(joint_lca_table, this_lca_table)
    end
    sort!(joint_lca_table, "contig_id")
    joint_lca_table = joint_lca_table[map(x -> x in viral_tax_ids, joint_lca_table[!, "taxon_id"]), :]

    # JUST VIRSORTER
    virsorter_score_tsv = joinpath(SRR_path, "virsorter", "final-viral-score.tsv")
    virsorter_results = Mycelia.parse_virsorter_score_tsv(virsorter_score_tsv)
    virsorter_results[!, "seqname"] = parse.(Int, first.(split.(virsorter_results[!, "seqname"], '|')))

    # JUST GENOMAD
    genomad_virus_summary = joinpath(SRR_path, "genomad", "final.contigs.fastg.gfa_summary", "final.contigs.fastg.gfa_virus_summary.tsv")
    genomad_results = DataFrames.DataFrame(uCSV.read(genomad_virus_summary, delim='\t', header=1, typedetectrows=100)...)

    number_of_hits = StatsBase.countmap(vcat(
        blast_hits_top_hits_table[!, "Crontig"],
        joint_lca_table[!, "contig_id"],
        virsorter_results[!, "seqname"],
        genomad_results[!, "seq_name"]
    ))

    majority_support_contigs = Set(keys(filter(x -> x[2] >= (n_methods/2), number_of_hits)))
    viral_blast_top_hits_table = blast_hits_top_hits_table[map(x -> x in majority_support_contigs, blast_hits_top_hits_table[!, "Contig"]), :]
    viral_blast_top_hits_table = viral_blast_top_hits_table[!, ["% identity", "alignment length"]]
    append!(viral_blast_hits_joint_table, viral_blast_top_hits_table)
end

KeyError: KeyError: key "usage_request" not found

Progress:   5%|██                                       |  ETA: 0:18:46

KeyError: KeyError: key "usage_request" not found

Progress:   5%|██▎                                      |  ETA: 0:22:03

KeyError: KeyError: key "usage_request" not found

...

Progress:  11%|████▌                                    |  ETA: 0:23:27

KeyError: KeyError: key "usage_request" not found

Progress:  36%|██████████████▊                          |  ETA: 0:14:37

KeyError: KeyError: key "usage_request" not found

Progress:  84%|██████████████████████████████████▎      |  ETA: 0:04:33

In [ ]:
p = StatsPlots.scatter(
    viral_blast_hits_joint_table[!, "alignment length"],
    viral_blast_hits_joint_table[!, "% identity"],
    title = "aligment quality for conensus viral contigs",
    xlabel = "alignment length",
    ylabel = "% identity",
    legend=false
)

now_string = replace(string(Dates.now()), r"[^\d+]" => "")
StatsPlots.savefig(p, joinpath(RESULTS_DIR, "figure-1.$(now_string).png"))
display(p)

In [ ]:
p = StatsPlots.scatter(
    viral_blast_hits_joint_table[!, "alignment length"],
    viral_blast_hits_joint_table[!, "% identity"],
    title = "aligment quality for conensus viral contigs",
    xlabel = "alignment length",
    ylabel = "% identity",
    legend=false
)

now_string = replace(string(Dates.now()), r"[^\d+]" => "")
StatsPlots.savefig(p, joinpath(RESULTS_DIR, "figure-1.$(now_string).png"))
display(p)

In [ ]:
# contig_info_table[!, "viral_classification_count"] = map(contig -> get(contig_support_counts, contig, 0), contig_info_table[!, "Contig"])
# contig_info_table[!, "viral_classification_percent"] = round.((contig_info_table[!, "viral_classification_count"] ./ n_methods) .* 100, digits=1)
# end
# sample_summary_table

In [ ]:
# SRR_paths
# SRR_path = rand(SRR_paths)

In [ ]:
# IMGVR_sam = joinpath(SRR_path, "megahit", "final.contigs.fastg.gfa.viral.fna.IMGVR_all_nucleotides-high_confidence.fna.gz.sam")

In [ ]:
# uCSV.read(IMGVR_sam, delim='\t', typedetectrows=100)

In [ ]:
# reader = open(XAM.SAM.Reader, IMGVR_sam)
# # Iterate over BAM records.
# for record in reader
#     # `record` is a BAM.Record object.
#     if XAM.SAM.ismapped(record)
#         # Print the mapped position.
#         println(XAM.SAM.refname(record), ':', XAM.SAM.position(record))
#     end
# end

# # Close the BAM file.
# close(reader)

Based on these results, we will go for a 3x confirmation via targetted blast where the sequence must have a high confidence match for a viral hit using:
- ref_viruses_rep_genomes_blastn
- ref_viruses_rep_genomes_dcmegablast
- nt_viral_validation_megablast

Full nt database attempts to validate viral contigs came back primarily with bacterial artificial chromosomes associated with human cell lines that did not seem like valuable hits to us. We also didn't expect to find *novel* viruses by megablasting against the ref_viruses_rep_genomes, given that it is a limited, representative set of all known viruses and the official description of the algorithm is "Traditional megablast used to find very similar (e.g., intraspecies or closely related species) sequences"
https://www.ncbi.nlm.nih.gov/books/NBK569839/

Because of this, we felt that getting a multiple redudant hits using the most flexibile algorithm (blastn) against the highest quality blast db (ref_viruses_rep_genomes), a less flexible, but still cross-species algorithm (dc-megablast: Discontiguous megablast used to find more distant (e.g., interspecies) sequences), and finally a 3rd validation hit against a potentially lower quality due to less manual curation viral database (nt_viral) using the strictest alogrithm, megablast.

Because of the low concordance of Kraken classifications to blast classifications, we consider the calls with much less weight than the classified contigs, but include them because the data was generated and it may prove informative to others.

As an update to the above, we found that we were unable to perform the `nt_viral_validation_megablast` in a reasonable amount of time, so we will be dropping that requirement.

Instead, we will perform a final re-assembly and then perform the final validation blast and protein-level classification on that final assembly

The following was a manual, indepth analysis of the first sample that we used to inform our validation approach

In [ ]:
# # extract contigs that came back as viral in the targetted screen
# # blast back against NCBI nt and confirm they are still viral
# SRR_path = first(SRR_paths)
# SRR = basename(SRR_path)

# # ref_viruses_ref_genomes_blast_report = joinpath(SRR_path, "blastn", "final.contigs.fastg.gfa.fna.blastn.ref_viruses_rep_genomes.blastn.txt")
# # ref_viruses_ref_genomes_blast_results = Mycelia.parse_blast_report(ref_viruses_ref_genomes_blast_report)
# # possible_viral_contigs = Set(unique(ref_viruses_ref_genomes_blast_results[!, "query id"]))
# # assembled_fasta = joinpath(SRR_path, "megahit", "final.contigs.fastg.gfa.fna")
# # viral_fasta = replace(assembled_fasta, ".fna" => ".potential_viral_contigs.fna")
# # potential_viral_records = filter(x -> FASTX.identifier(x) in possible_viral_contigs, collect(Mycelia.open_fastx(assembled_fasta)))
# # open(viral_fasta, "w") do io
# #     fastx_io = FASTX.FASTA.Writer(io)
# #     for record in potential_viral_records
# #         write(fastx_io, record)
# #     end
# #     close(fastx_io)
# # end

# # potential_viral_records


# taxon_id_to_kingdom_map = Dict{Int, String}()
# kingdom_to_taxon_id_map = Dict(
#     "Viruses" => 10239,
#     "Archaea" => 2157,
#     "Bacteria" => 2,
#     "Eukaryota" => 2759,
#     "Other" => 28384,
#     "Unclassified" => 12908
# )
# ProgressMeter.@showprogress for (kingdom, taxon_id) in kingdom_to_taxon_id_map
#     for child_taxon_id in parse.(Int, filter(!isempty, readlines(`taxonkit list --data-dir $(taxdump_dir) --ids $(taxon_id) --indent=""`)))
#         taxon_id_to_kingdom_map[child_taxon_id] = kingdom
#     end
# end

# ref_viruses_rep_genomes_blastn_contigs = unique(ref_viruses_rep_genomes_blastn_results[!, "query id"])

# ref_viruses_rep_genomes_dcmegablast_contigs = unique(ref_viruses_rep_genomes_dcmegablast_results[!, "query id"])

# # # 8 non-overlapping hits!
# # union(ref_viruses_rep_genomes_dcmegablast_contigs, ref_viruses_rep_genomes_blastn_contigs)
# # # these all have pretty low e-values, I'm not going to worry about them
# # dcmegablast_only = setdiff(ref_viruses_rep_genomes_dcmegablast_contigs, ref_viruses_rep_genomes_blastn_contigs)
# # ref_viruses_rep_genomes_dcmegablast_results[map(x -> x in dcmegablast_only, ref_viruses_rep_genomes_dcmegablast_results[!, "query id"]), :]

# ref_viruses_rep_genomes_megablast_contigs = unique(ref_viruses_rep_genomes_megablast_results[!, "query id"])
# # nt_viral_validation_megablast_results
# # nt_validation_megablast_results
# # all megablast hits are subset of blastn hits
# # union(ref_viruses_rep_genomes_megablast_contigs, ref_viruses_rep_genomes_blastn_contigs)

# nt_viral_validation_megablast_contigs = unique(nt_viral_validation_megablast_results[!, "query id"])
# # nt_validation_megablast_results

# # union(nt_viral_validation_megablast_contigs, ref_viruses_rep_genomes_blastn_contigs)
# # map(x -> x in viral_taxon_ids, nt_viral_validation_megablast_results[!, "subject tax id"])

# taxon_id_to_kingdom_map

# # only 12 contigs are still considered viral contigs after mapping to nt complete, but the hits aren't very convicing (bacterial artificial chromosomes?)
# nt_validation_megablast_contigs = unique(nt_validation_megablast_results[map(x -> get(taxon_id_to_kingdom_map, x, "") == "Viruses", nt_validation_megablast_results[!, "subject tax id"]), "query id"])

# full_contig_set = union(nt_validation_megablast_contigs, nt_viral_validation_megablast_contigs, ref_viruses_rep_genomes_blastn_contigs, ref_viruses_rep_genomes_dcmegablast_contigs, ref_viruses_rep_genomes_megablast_contigs)

# # get all of the reads mapping to each

# bam_file = joinpath(SRR_path, "megahit", "final.contigs.fastg.gfa.fna.bwa.bam")
# # bamfile = first(filter(x -> occursin(r"\.bam$", x), readdir(joinpath(SRR_dir, "megahit"), join=true)))

# # implement as the following nested dictionary
# # contigs => reads => taxon_id

# function generate_contig_to_reads_map(bamfile, contigs_of_interest)
#     contigs_to_reads_map = Dict(contig => Set{String}() for contig in contigs_of_interest)
#     # reads_of_interest = Set{String}()
#     reader = open(XAM.BAM.Reader, bamfile)
#     for record in reader
#         if XAM.BAM.ismapped(record) && (XAM.BAM.refname(record) in contigs_of_interest)
#             push!(contigs_to_reads_map[XAM.BAM.refname(record)], XAM.BAM.tempname(record))
#         end
#     end
#     close(reader)
#     return contigs_to_reads_map
# end

# # 1200 seconds
# @time contigs_to_reads_map = generate_contig_to_reads_map(bam_file, full_contig_set)

# function read_kraken_output(kraken_output)
#     # read_kraken_report
#     header = [
#         "classification status",
#         "sequence ID",
#         "taxon ID",
#         "sequence length",
#         "LCA mappings"
#     ]
#     data, _ = uCSV.read(IOBuffer(join(filtered_lines, '\n')), delim='\t')
#     return DataFrames.DataFrame(data, header)
# end

# # readdir(joinpath(SRR_path, "kraken"))

# reads_of_interest = reduce(union, values(contigs_to_reads_map))

# kraken_output = last(filter(x -> occursin(r"\.kraken-output\.tsv", x), readdir(joinpath(SRR_path, "kraken"), join=true)))
# if occursin(r"\.gz", kraken_output)
#     kraken_buffer = open(`gzip -dc $(kraken_output)`)
# else
#     kraken_buffer = open(kraken_output)
# end

# filtered_lines = String[]
# for line in eachline(kraken_buffer)
#     split_line = split(line, '\t')
#     if split_line[2] in reads_of_interest
#         push!(filtered_lines, line)
#     end
# end
# close(kraken_buffer)

# read_classifications = read_kraken_output(IOBuffer(join(filtered_lines, '\n')))
# read_classifications[!, "parsed taxon ID"] = map(x -> match(r"\(taxid (\d+)\)", x).captures[1], read_classifications[!, "taxon ID"])
# read_classifications

# read_classifications_map = Dict(row["sequence ID"] => parse(Int, row["parsed taxon ID"]) for row in DataFrames.eachrow(read_classifications))

# contigs_to_taxon_counts_map = Dict()
# for (contig, reads) in contigs_to_reads_map
#     contigs_to_taxon_counts_map[contig] = StatsBase.countmap(get(taxon_id_to_kingdom_map, read_classifications_map[read], "Unclassified") for read in reads)
# end
# contigs_to_taxon_counts_map

# contigs_to_taxon_proportions_map = Dict()
# for (contig, taxon_counts) in contigs_to_taxon_counts_map
#     total_count = sum(values(taxon_counts))
#     contigs_to_taxon_proportions_map[contig] = Dict(kingdom => count / total_count for (kingdom, count) in taxon_counts)
# end
# contigs_to_taxon_proportions_map

# # ref_viruses_rep_genomes_blastn_results
# # ref_viruses_rep_genomes_dcmegablast_results
# # ref_viruses_rep_genomes_megablast_results
# # nt_viral_validation_megablast_results
# # nt_validation_megablast_results

# contig_classification_results = 
# DataFrames.DataFrame(
#     union(
#         DataStructures.OrderedDict("Contig" => String[]),
#         DataStructures.OrderedDict(k => Float64[] for k in keys(kingdom_to_taxon_id_map)),
#         # these are ordered by most hits to fewest hits
#         DataStructures.OrderedDict(db_algorithm => Bool[] for db_algorithm in ["ref_viruses_rep_genomes_blastn", "ref_viruses_rep_genomes_dcmegablast", "nt_viral_validation_megablast", "ref_viruses_rep_genomes_megablast", "nt_validation_megablast"])
#     )
# )

# for (contig, taxon_proportions) in contigs_to_taxon_proportions_map
#     row = Dict{Any, Any}("Contig" => contig)
#     for k in keys(kingdom_to_taxon_id_map)
#         row[k] = get(taxon_proportions, k, 0.0)
#     end
#     row["ref_viruses_rep_genomes_blastn"] = contig in ref_viruses_rep_genomes_blastn_contigs
#     row["nt_viral_validation_megablast"] = contig in nt_viral_validation_megablast_contigs
#     row["ref_viruses_rep_genomes_dcmegablast"] = contig in ref_viruses_rep_genomes_dcmegablast_contigs
#     row["ref_viruses_rep_genomes_megablast"] = contig in ref_viruses_rep_genomes_megablast_contigs
#     row["nt_validation_megablast"] = contig in nt_validation_megablast_contigs
#     push!(contig_classification_results, row)
# end
# contig_classification_results[!, "top_kingdom"] .= ""
# for (i, row) in enumerate(DataFrames.eachrow(contig_classification_results))
#     max_hit = ""
#     max_value = 0.0
#     for k in keys(kingdom_to_taxon_id_map)
#         if row[k] > max_value
#             max_value = row[k]
#             max_hit = k
#         end
#     end
#     contig_classification_results[i, "top_kingdom"] = max_hit
# end

# m = Int.(Matrix(
#     contig_classification_results[!, [
#         "ref_viruses_rep_genomes_blastn",
#         "ref_viruses_rep_genomes_dcmegablast",
#         "nt_viral_validation_megablast",
#         "ref_viruses_rep_genomes_megablast",
#         "nt_validation_megablast"
#     ]
# ]))

# contig_classification_results[!, "blast_hits"] = map(r -> sum(r), eachrow(m))

# contig_classification_results

# # eukaryotic top hit - kraken is asserting these are human contamination
# sum(contig_classification_results[!, "Eukaryota"])

# # novel sequences are second hit, this is exciting!
# sum(contig_classification_results[!, "Unclassified"])

# sum(contig_classification_results[!, "Bacteria"])

# sum(contig_classification_results[!, "Other"])

# sum(contig_classification_results[!, "Viruses"])

# sum(contig_classification_results[!, "Archaea"])

# contig_classification_results_summary = contig_classification_results[!, ["Contig", "top_kingdom", "blast_hits"]]

# contig_classification_results_summary

# StatsBase.countmap(contig_classification_results_summary[!, "blast_hits"])

# contig_classification_results_summary[contig_classification_results_summary[!, "blast_hits"] .== 5, :]